# 【問題1】機械翻訳の実行とコードリーディング

In [2]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 16
Max sequence length for outputs: 59
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 59s 7ms/step - loss: 1.1640 - accuracy: 0.7253 - val_loss: 1.0481 - val_accuracy: 0.7016
Epoch 2/100
8000/8000 [==============================] - 51s 6ms/step - loss: 0.8519 - accuracy: 0.7691 - val_loss: 0.8334 - val_accuracy: 0.7699
Epoch 3/100
8000/8000 [==============================] - 52s 7ms/step - loss: 0.6841 - accuracy: 0.8079 - val_loss: 0.7226 - val_accuracy: 0.7890
Epoch 4/100
8000/8000 [==============================] - 61s 8ms/step - loss: 0.5979 - accuracy: 0.8270 - val_loss: 0.6753 - val_accuracy: 0.8049
Epoch 5/100
8000/8000 [====

-
Input sentence: Wait!
Decoded sentence: Attandez !

-
Input sentence: Wait!
Decoded sentence: Attandez !

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Hello!
Decoded sentence: Salut !

-
Input sentence: Hello!
Decoded sentence: Salut !

-
Input sentence: I see.
Decoded sentence: Je comprends.

-
Input sentence: I try.
Decoded sentence: J'essaye.

-
Input sentence: I won!
Decoded sentence: Je l'ai emporté !

-
Input sentence: I won!
Decoded sentence: Je l'ai emporté !

-
Input sentence: I won.
Decoded sentence: Je suis mon tiand.

-
Input sentence: Oh no!
Decoded sentence: Oh non !

-
Input sentence: Attack!
Decoded sentence: Attaque !

-
Input sentence: Attack!
Decoded sentence: Attaque !

-
Input sentence: Cheers!
Decoded sentence: À votre santé !

-
Input sentence: Cheers!
Decoded sentence: À votre santé !

-
Input sentence: Cheers!
Decoded se

- 21行目から40行目でインプットの文字と予測する文字をリストに入れ、その数を計算している。また、一文の文字数の最大値を計算している。
- 48行目から51行目で文字にidをふっている。
- 53行目から61行目で[サンプル数, 文の語彙数, 文字の種類数]の空の入れ物を作っている
- 63行目から75行目でone-hotエンコーディングをしている。
- 77行目から81行目でエンコーダーのモデル化をしている。
- 84行目から92行目でデコーダーのモデル化をしている。
- 96行目から106行目でseq2seqのモデル化、学習、モデルの保存を行っている。
- 117行目から128行目でpredict用のモデル化を行っている。
- 132行目から135行目でトークンを文字に戻す用の辞書を作成している。
- 138行目から173行目でseq2seqによりinputの文章をoutputの文章に変換している。
- 176行目から183行目でinputの文章を1文ずつ変換し、結果をprintしている。

# 【問題2】イメージキャプショニングの学習済みモデルの実行

In [12]:
cd ../../../../

C:\Users\shota\diveintocode-ml\Sprint24


In [13]:
cd coco/PythonAPI/

C:\Users\shota\diveintocode-ml\Sprint24\coco\PythonAPI


In [5]:
!make

'make' は、内部コマンドまたは外部コマンド、
操作可能なプログラムまたはバッチ ファイルとして認識されていません。


In [6]:
!python setup.py build

running build
running build_py
running build_ext
skipping 'pycocotools\_mask.c' Cython extension (up-to-date)


In [7]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing pycocotools.egg-info\PKG-INFO
writing dependency_links to pycocotools.egg-info\dependency_links.txt
writing requirements to pycocotools.egg-info\requires.txt
writing top-level names to pycocotools.egg-info\top_level.txt
reading manifest file 'pycocotools.egg-info\SOURCES.txt'
writing manifest file 'pycocotools.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
running build_py
running build_ext
skipping 'pycocotools\_mask.c' Cython extension (up-to-date)
creating build\bdist.win-amd64\egg
creating build\bdist.win-amd64\egg\pycocotools
copying build\lib.win-amd64-3.7\pycocotools\coco.py -> build\bdist.win-amd64\egg\pycocotools
copying build\lib.win-amd64-3.7\pycocotools\cocoeval.py -> build\bdist.win-amd64\egg\pycocotools
copying build\lib.win-amd64-3.7\pycocotools\mask.py -> build\bdist.win-amd64\egg\pycocotools
copying build\lib.win-amd64-3.7\pycocotools\_mask.cp37-win_amd64.pyd -> b

zip_safe flag not set; analyzing archive contents...
pycocotools.__pycache__._mask.cpython-37: module references __file__
error removing c:\anaconda\lib\site-packages\pycocotools-2.0-py3.7-win-amd64.egg: [WinError 5] アクセスが拒否されました。: 'c:\\anaconda\\lib\\site-packages\\pycocotools-2.0-py3.7-win-amd64.egg\\pycocotools\\_mask.cp37-win_amd64.pyd'
error removing c:\anaconda\lib\site-packages\pycocotools-2.0-py3.7-win-amd64.egg: [WinError 145] ディレクトリが空ではありません。: 'c:\\anaconda\\lib\\site-packages\\pycocotools-2.0-py3.7-win-amd64.egg\\pycocotools'
error removing c:\anaconda\lib\site-packages\pycocotools-2.0-py3.7-win-amd64.egg: [WinError 145] ディレクトリが空ではありません。: 'c:\\anaconda\\lib\\site-packages\\pycocotools-2.0-py3.7-win-amd64.egg'
error: [Errno 13] Permission denied: 'c:\\anaconda\\lib\\site-packages\\pycocotools-2.0-py3.7-win-amd64.egg\\pycocotools\\_mask.cp37-win_amd64.pyd'


In [16]:
cd ../../

C:\Users\shota\diveintocode-ml\Sprint24


In [17]:
cd pytorch-tutorial/tutorials/03-advanced/image_captioning/

C:\Users\shota\diveintocode-ml\Sprint24\pytorch-tutorial\tutorials\03-advanced\image_captioning


In [18]:
!pip install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"

In [19]:
!python sample.py --image='png/example.png'

Traceback (most recent call last):
  File "sample.py", line 8, in <module>
    from build_vocab import Vocabulary
  File "C:\Users\shota\diveintocode-ml\Sprint24\pytorch-tutorial\tutorials\03-advanced\image_captioning\build_vocab.py", line 5, in <module>
    from pycocotools.coco import COCO
ModuleNotFoundError: No module named 'pycocotools.coco'


## 申し訳ございません。エラーが解消できず、ここまでしかできておりません。

# 【問題3】Kerasで動かしたい場合はどうするかを調査

## pytorchの学習済みモデルをonnx形式で保存し、それをkerasで読み込む。